In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
games = pd.read_parquet('../Datasets/gamesoh.parquet')
reviews = pd.read_parquet('../Datasets/reviews.parquet')
items = pd.read_parquet('../Datasets/itemso.parquet')

generos = list(games.drop(columns=['item_id']).columns)
gnames = pd.read_parquet('../Datasets/games.parquet')
fnames = pd.read_parquet('../Datasets/fnames.parquet')
unames = pd.read_parquet('../Datasets/unames.parquet')

lid=list(set(gnames['item_id']))
lfid=list(set(fnames['item_id']))
lanio=list(set(reviews['posted'].to_list()))

In [16]:
# Obtener el uso de la memoria por columna
m1 = games.memory_usage(deep=True)
m2 = reviews.memory_usage(deep=True)
m3 = items.memory_usage(deep=True)
m4 = gnames.memory_usage(deep=True)
m5 = fnames.memory_usage(deep=True)
m6 = unames.memory_usage(deep=True)
# Sumar el uso total de la memoria
t1 = m1.sum() / (1024 ** 2)  # Convertir a megabytes
t2 = m2.sum() / (1024 ** 2)  # Convertir a megabytes
t3 = m3.sum() / (1024 ** 2)  # Convertir a megabytes
t4 = m4.sum() / (1024 ** 2)  # Convertir a megabytes
t5 = m5.sum() / (1024 ** 2)  # Convertir a megabytes
t6 = m6.sum() / (1024 ** 2)  # Convertir a megabytes

print(f"Uso de la memoria para games:   {t1:.2f} MB")
print(f"Uso de la memoria para reviews: {t2:.2f} MB")
print(f"Uso de la memoria para items:   {t3:.2f} MB")
print(f"Uso de la memoria para gnames:  {t4:.2f} MB")
print(f"Uso de la memoria para fnames:  {t5:.2f} MB")
print(f"Uso de la memoria para unames:  {t6:.2f} MB")
print(f"Uso de la memoria en total:     {t1+t2+t3+t4+t5+t6:.2f} MB")

Uso de la memoria para games:   1.85 MB
Uso de la memoria para reviews: 1.24 MB
Uso de la memoria para items:   59.48 MB
Uso de la memoria para gnames:  3.04 MB
Uso de la memoria para fnames:  0.04 MB
Uso de la memoria para unames:  6.34 MB
Uso de la memoria en total:     72.00 MB


In [17]:
def PlayTimeGenre(genero):
  genero=genero.upper()
  if genero in generos:
    lgames = games.loc[games[genero] == 1, ['item_id']]
    lhoras = items.groupby('item_id')['playtime_forever'].sum().reset_index()
    dfplay = pd.merge(lgames, lhoras, on='item_id', how='inner')
    dfplay = pd.merge(dfplay, gnames, on='item_id', how='inner')
    df_horas = dfplay.groupby('release_date')['playtime_forever'].sum().reset_index()
    df_horas_ordenado = df_horas.sort_values(by='playtime_forever', ascending=False)
    resultado=df_horas_ordenado.head(1)
    prin = "Año de lanzamiento con más horas jugadas para Género " + genero
    anio = int(resultado['release_date'].iloc[0])
    return {prin : anio}
  else:
    return {"El Genero indicado no se encuentra en la base de datos":genero}

In [18]:
PlayTimeGenre("2d")

{'Año de lanzamiento con más horas jugadas para Género 2D': 2011}

In [20]:
def UserForGenre(genero:str):
    genero=genero.upper()
    if genero in generos:
        dfgames = games[['item_id',genero]]
        dfgames = dfgames[games[genero]==1]
        dfuserg = pd.merge(dfgames,items,on='item_id',how='inner')
        uhoras = dfuserg.groupby('user_id')['playtime_forever'].sum().reset_index()
        uhoras = uhoras.sort_values(by='playtime_forever', ascending=False)
        uhoras = uhoras.head(1)
        uhoras = uhoras['user_id'].iloc[0]
        dfuserg = dfuserg[dfuserg['user_id']==uhoras]
        dfuserg = pd.merge(dfuserg,gnames,on='item_id',how='inner')
        dfuserg = dfuserg.groupby('release_date')['playtime_forever'].sum().reset_index()
        dfuserg = dfuserg.sort_values(by='release_date', ascending=False)
        dfuserg = dfuserg[dfuserg['playtime_forever']>0]
        dfuserg['playtime_forever'] = dfuserg['playtime_forever'].astype(int)
        dfuserg = dfuserg[['playtime_forever', 'release_date']]
        dfuserg.rename(columns={'release_date': 'Año', 'playtime_forever':'Horas'}, inplace=True)
        #di = dfuserg.apply(lambda row: row.to_dict(), axis=1).tolist()
        di=dfuserg.to_dict(orient='records')
        uhoras = unames[unames['user_id']==uhoras]
        uhoras = uhoras['user_name'].iloc[0]
        prin="Usuario con más horas jugadas para Género " + genero
        return {prin:uhoras,"Horas Jugadas":di}
    else:
        return {"El Genero indicado no se encuentra en la base de datos ":genero}

In [22]:
UserForGenre('AcTiOn')

{'Usuario con más horas jugadas para Género ACTION': 'Sp3ctre',
 'Horas Jugadas': [{'Horas': 43327, 'Año': 2017},
  {'Horas': 30195, 'Año': 2016},
  {'Horas': 394314, 'Año': 2015},
  {'Horas': 132864, 'Año': 2014},
  {'Horas': 122158, 'Año': 2013},
  {'Horas': 384238, 'Año': 2012},
  {'Horas': 155459, 'Año': 2011},
  {'Horas': 82766, 'Año': 2010},
  {'Horas': 108771, 'Año': 2009},
  {'Horas': 8533, 'Año': 2008},
  {'Horas': 112794, 'Año': 2007},
  {'Horas': 90253, 'Año': 2006},
  {'Horas': 21339, 'Año': 2005},
  {'Horas': 127411, 'Año': 2004},
  {'Horas': 7744, 'Año': 2003},
  {'Horas': 238, 'Año': 2002},
  {'Horas': 250, 'Año': 2001},
  {'Horas': 70644, 'Año': 2000},
  {'Horas': 44, 'Año': 1999},
  {'Horas': 242, 'Año': 1997},
  {'Horas': 217, 'Año': 1995}]}

In [23]:
def UsersRecommend(anio):
  if anio in lanio:
    dfa = reviews[(reviews['posted'] == anio) & (reviews['recommend'] == True) & (reviews['sentiment_analysis'] >0)].copy()
    dfa = dfa.groupby('item_id')['sentiment_analysis'].sum().reset_index()
    dfa = dfa.sort_values('sentiment_analysis', ascending=False)
    dfa['item_id'] = dfa['item_id'].astype(int)
    top_games = dfa.head(3)['item_id'].to_list()
    for i,g in enumerate(top_games):
        if g in lid:
            top_games[i]=gnames[gnames['item_id'] == top_games[i]].head(1)['title'].iloc[0]
        else:
            top_games[i]=fnames[fnames['item_id'] == top_games[i]].head(1)['item_name'].iloc[0]
    return{'Puesto 1':top_games[0],'Puesto 2':top_games[1],'Puesto 3':top_games[2]}
  else:
    return {f'El año indicado no se encuentra en la base de datos: ':anio}

In [24]:
UsersRecommend(2013)

{'Puesto 1': 'Team Fortress 2',
 'Puesto 2': "Garry's Mod",
 'Puesto 3': 'Dota 2'}

In [25]:
def UsersNotRecommend(anio):
  if anio in lanio:
    dfa = reviews[(reviews['posted'] == anio) & (reviews['recommend'] == False) & (reviews['sentiment_analysis'] ==0)].copy()
    dfa['sentiment_analysis']=1
    dfa = dfa.groupby('item_id')['sentiment_analysis'].sum().reset_index()
    dfa['item_id'] = dfa['item_id'].astype(int)
    dfa = dfa.sort_values('sentiment_analysis', ascending=False)
    top_games = dfa.head(3)['item_id'].to_list()
    for i,g in enumerate(top_games):
      if g in lid:
        top_games[i]=gnames[gnames['item_id'] == top_games[i]].head(1)['title'].iloc[0]
      else:
        top_games[i]=fnames[fnames['item_id'] == top_games[i]].head(1)['item_name'].iloc[0]
    if len(top_games)<2:
      return{'Puesto 1':top_games[0],'Puesto 2':'','Puesto 3':''}
    elif len(top_games)<3:
      return{'Puesto 1':top_games[0],'Puesto 2':top_games[1],'Puesto 3':''}
    else:
      return{'Puesto 1':top_games[0],'Puesto 2':top_games[1],'Puesto 3':top_games[2]}
  else:
    return {f'El año indicado no se encuentra en la base de datos: ':anio}

In [26]:
UsersNotRecommend(2013)

{'Puesto 1': 'Team Fortress 2',
 'Puesto 2': 'Call of Duty®: Ghosts',
 'Puesto 3': 'Half-Life: Before'}

In [49]:
def sentiment_analysis(anio):
    dfs = pd.merge(reviews[['item_id','sentiment_analysis']],gnames[['item_id','release_date']],on='item_id',how='inner')
    dfs = dfs[dfs['release_date']==anio]
    dfs = dfs[['sentiment_analysis']]
    dfs = dfs.value_counts().reset_index()
    dfs = dfs.sort_values('sentiment_analysis', ascending=True)
    dfs = dfs['count'].to_list()
    return{'Negative':dfs[0],'Neutral':dfs[1],'Positive':dfs[2]}

In [50]:
sentiment_analysis(2014)

{'Negative': 1040, 'Neutral': 1067, 'Positive': 4341}

In [74]:
def recomendacion_juego(idg):
    if (idg in lid) or (idg in lfid):
        gx=pd.DataFrame(games.loc[idg])
        gx=pd.pivot_table(gx,columns=['item_id', '2D', 'ACTION', 'ADVENTURE', 'ANIME', 'ARCADE',
       'ATMOSPHERIC', 'CASUAL', 'CLASSIC', 'CO-OP', 'COMEDY', 'CUTE',
       'DESIGN & ILLUSTRATION', 'DIFFICULT', 'EARLY ACCESS', 'EXPLORATION',
       'FPS', 'FAMILY FRIENDLY', 'FANTASY', 'FEMALE PROTAGONIST',
       'FIRST-PERSON', 'FREE TO PLAY', 'FUNNY', 'GORE', 'GREAT SOUNDTRACK',
       'HIDDEN OBJECT', 'HORROR', 'INDIE', 'LOCAL CO-OP', 'LOCAL MULTIPLAYER',
       'MASSIVELY MULTIPLAYER', 'MOVIE', 'MULTIPLAYER', 'ONLINE CO-OP',
       'OPEN WORLD', 'PIXEL GRAPHICS', 'PLATFORMER', 'POINT & CLICK', 'PUZZLE',
       'RPG', 'RPGMAKER', 'RACING', 'RETRO', 'ROGUE-LIKE', 'SANDBOX', 'SCI-FI',
       "SHOOT 'EM UP", 'SHOOTER', 'SIMULATION', 'SINGLEPLAYER', 'SPACE',
       'SPORTS', 'STORY RICH', 'STRATEGY', 'SURVIVAL', 'TACTICAL',
       'THIRD PERSON', 'TURN-BASED', 'UTILITIES', 'VIOLENT', 'VISUAL NOVEL',
       'ZOMBIES'])
        gx.drop(columns=['item_id'], inplace=True)
        xgames=games[games['item_id']!=idg].copy()
        xgames=xgames.reset_index()
        xgames.drop(columns=['index'],inplace=True)
        ygames=xgames.copy()
        xgames.drop(columns=['item_id'],inplace=True)
        matriz = cosine_similarity(gx,xgames)
        matrix = pd.DataFrame(matriz)
        matriz=matriz[0]
        reco=[]
        for i in range(5):
          imax=np.argmax(matriz)
          reco.append(imax)
          matriz[imax]=0
        top_games = []
        for i in reco:
          top_games.append(ygames.iloc[i]['item_id'])
        for i,g in enumerate(top_games):
          if g in lid:
            top_games[i]=gnames[gnames['item_id'] == top_games[i]]['title'].iloc[0]
          else:
            top_games[i]=fnames[fnames['item_id'] == top_games[i]]['item_name'].iloc[0]
        return {"juegos recomendados":
                 [{"Puesto 1":top_games[0]},
                  {"Puesto 2":top_games[1]},
                  {"Puesto 3":top_games[2]},
                  {"Puesto 4":top_games[3]},
                  {"Puesto 5":top_games[4]}]}
    else:
        return {"id de juego no se encuentra en la base de datos ":idg}

In [75]:
recomendacion_juego(440)

{'juegos recomendados': [{'Puesto 1': 'Big Money! Deluxe'},
  {'Puesto 2': 'Hammer Heads Deluxe'},
  {'Puesto 3': 'Iggle Pop Deluxe'},
  {'Puesto 4': 'Pizza Frenzy Deluxe'},
  {'Puesto 5': 'Typer Shark! Deluxe'}]}